В этом задании мы реализуем другую модель машинного обучения - линейный классификатор. Линейный классификатор подбирает для каждого класса веса, на которые нужно умножить значение каждого признака и потом сложить вместе. Тот класс, у которого эта сумма больше, и является предсказанием модели.
Задание 1.2 - Линейный классификатор (Linear classifier)

В этом задании вы:

потренируетесь считать градиенты различных многомерных функций
реализуете подсчет градиентов через линейную модель и функцию потерь softmax
реализуете процесс тренировки линейного классификатора
подберете параметры тренировки на практике
На всякий случай, еще раз ссылка на туториал по numpy:
http://cs231n.github.io/python-numpy-tutorial/

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

from dataset import load_svhn, random_split_train_val
from gradient_check import check_gradient
from metrics import multiclass_accuracy 
import linear_classifer

In [2]:
def prepare_for_linear_classifier(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    # Add another channel with ones as a bias term
    train_flat_with_ones = np.hstack([train_flat, np.ones((train_X.shape[0], 1))])
    test_flat_with_ones = np.hstack([test_flat, np.ones((test_X.shape[0], 1))])    
    return train_flat_with_ones, test_flat_with_ones
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_linear_classifier(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

/tmp/ipykernel_4659/3741198026.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
/tmp/ipykernel_4659/3741198026.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0


In [3]:
# TODO: Implement check_gradient function in gradient_check.py
# All the functions below should pass the gradient check

def square(x):
    return float(x*x), 2*x

check_gradient(square, np.array([3.0]))

def array_sum(x):
    assert x.shape == (2,), x.shape
    return np.sum(x), np.ones_like(x)

check_gradient(array_sum, np.array([3.0, 2.0]))

def array_2d_sum(x):
    assert x.shape == (2,2)
    return np.sum(x), np.ones_like(x)

check_gradient(array_2d_sum, np.array([[3.0, 2.0], [1.0, 0.0]]))

CHECK GRADIENT
prediction is 
 [3.]
analytic grad is 
 [6.]


IndexError: tuple index out of range

In [ ]:
a = np.array([[3,2,4,5], [3,3,2,1]])
b = np.array([3,2,90,5])
for elem in b:
    print(elem)

check = np.isclose(a, b, atol=0.5)
print(check)


it = np.nditer(a, flags=['multi_index'], op_flags=['readwrite'])
ix = it.multi_index
it.iternext()
it.iternext()
it.iternext()
it.iternext()
# it.()
print(it.multi_index)

a -= np.max(a)
print("new a is ", a)

3
2
90
5
[[ True  True False  True]
 [ True False False False]]
(1, 0)
new a is  [[-2 -3 -1  0]
 [-2 -2 -3 -4]]


In [ ]:
# TODO Implement softmax and cross-entropy for single sample
probs = linear_classifer.softmax(np.array([-10, 0, 10]))

# Make sure it works for big numbers too!
probs = linear_classifer.softmax(np.array([1000, 0, 0]))
assert np.isclose(probs[0], 1.0)

orig predictions 
 [-10   0  10]
batch_size is  3
dimension is  1
probabilites are 
 [2.06106005e-09 4.53978686e-05 9.99954600e-01]
orig predictions 
 [1000    0    0]
batch_size is  3
dimension is  1
probabilites are 
 [1. 0. 0.]


In [ ]:
probs = linear_classifer.softmax(np.array([-5, 0, 5]))
linear_classifer.cross_entropy_loss(probs, np.array([[1]]))

orig predictions 
 [-5  0  5]
batch_size is  3
dimension is  1
probabilites are 
 [4.50940412e-05 6.69254912e-03 9.93262357e-01]
targ ind is  [[1]]
1
probs  (3,)
loss is  [[5.00676044]]


array([[5.00676044]])

In [ ]:
# TODO Implement combined function or softmax and cross entropy and produces gradient
loss, grad = linear_classifer.softmax_with_cross_entropy(np.array([1, 0, 0]), np.array([[1]]))
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, np.array([[1]])), np.array([1, 0, 0], float))

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# TODO Extend combined function so it can receive a 2d array with batch of samples
np.random.seed(42)
# Test batch_size = 1
num_classes = 4
batch_size = 1
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(float)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(int)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)
print("END")

# Test batch_size = 3
num_classes = 4
batch_size = 3
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(float)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(int)
# print("target_index is \n", target_index)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)
print("END")

# Make sure maximum subtraction for numberic stability is done separately for every sample in the batch
probs = linear_classifer.softmax(np.array([[20,0,0], [1000, 0, 0]]))
assert np.all(np.isclose(probs[:, 0], 1.0))
print("END")

CHECK GRADIENT
prediction is 
 [[ 1.  2. -1.  1.]]
loss array 
 [3.57972422]
analytic grad is 
 [[ 0.20603191  0.56005279 -0.97211661  0.20603191]]
x shape[1]  4
loss array 
 [3.57972628]
loss array 
 [3.57972216]
loss array 
 [3.57972982]
loss array 
 [3.57971862]
loss array 
 [3.5797145]
loss array 
 [3.57973394]
loss array 
 [3.57972628]
loss array 
 [3.57972216]
numeric grad array is 
 [[ 0.20603191  0.56005279 -0.97211661  0.20603191]]
Gradient check passed!
END
CHECK GRADIENT
prediction is 
 [[ 2. -1. -1.  1.]
 [ 0.  1.  1.  1.]
 [ 1.  2. -1.  2.]]
loss array 
 [1.38352864 1.2142833  3.88280282]
analytic grad is 
 [[ 0.68145256  0.03392753  0.03392753 -0.74930761]
 [ 0.10923177  0.29692274  0.29692274 -0.70307726]
 [ 0.15216302  0.41362198 -0.97940697  0.41362198]]
x shape[1]  4
loss array 
 [1.38353545 1.2142833  3.88280282]
loss array 
 [1.38352182 1.2142833  3.88280282]
loss array 
 [1.38352898 1.2142833  3.88280282]
loss array 
 [1.3835283  1.2142833  3.88280282]
loss array 


In [8]:
# TODO Implement linear_softmax function that uses softmax with cross-entropy for linear classifier
batch_size = 2
num_classes = 2
num_features = 3
np.random.seed(42)
W = np.random.randint(-1, 3, size=(num_features, num_classes)).astype(float)
X = np.random.randint(-1, 3, size=(batch_size, num_features)).astype(float)
target_index = np.ones(batch_size, dtype=int)

loss, dW = linear_classifer.linear_softmax(X, W, target_index)
check_gradient(lambda w: linear_classifer.linear_softmax(X, w, target_index), W)

loss array 
 [2.12692801 0.04858735]
CHECK GRADIENT
prediction is 
 [[ 1.  2.]
 [-1.  1.]
 [ 1.  2.]]
loss array 
 [2.12692801 0.04858735]
analytic grad is 
 [[-0.44039854  0.44039854]
 [-0.4166856   0.4166856 ]
 [ 0.46411148 -0.46411148]]
x shape[1]  2
loss array 
 [2.1269192  0.04858735]
loss array 
 [2.12693682 0.04858735]
loss array 
 [2.12693682 0.04858735]
loss array 
 [2.1269192  0.04858735]
loss array 
 [2.1269192  0.04858783]
loss array 
 [2.12693682 0.04858688]
loss array 
 [2.12693682 0.04858688]
loss array 
 [2.1269192  0.04858783]
loss array 
 [2.12693682 0.04858783]
loss array 
 [2.1269192  0.04858688]
loss array 
 [2.1269192  0.04858688]
loss array 
 [2.12693682 0.04858783]
numeric grad array is 
 [[-0.44039854  0.44039854]
 [-0.4166856   0.4166856 ]
 [ 0.46411148 -0.46411148]]
it is  -0.4403985389922482
it is  0.4403985389922482
it is  -0.4166856024112597
it is  0.4166856024112597
it is  0.4641114755732367
it is  -0.4641114755732367
Gradient check passed!


True

In [ ]:
# a = np.array([[[2,3,2,1], [3,42,4,1], [3,4,2,1]], [[2,3,2,1], [3,0,4,1], [3,4,2,1]], [[2,30,2,1], [3,1,4,1], [3,4,2,1]]])
# print(a)
# print("max is \n", np.max(a, axis=2))
# print(a.ndim)


b = np.array([[2,3,2,1], [3,42,4,1], [3,4,2,1]])
c = np.max(b, axis=1)
print("b is \n",b)
print ("c is \n", c)
for elem, i in zip(b, range(b.size)):
    elem -= c[i]
# print("f is ",f )
# b -= np.max(b, axis=1)
print("b is \n",b)
# probabilities = np.zeros_like(predictions)
# sum_exps = np.sum(np.exp(b))
# probabilities = np.exp(b)/sum_exps
# print("probs are ", probabilities)

dd = np.array([[3, 3]])
print(len(dd))


b is 
 [[ 2  3  2  1]
 [ 3 42  4  1]
 [ 3  4  2  1]]
c is 
 [ 3 42  4]
b is 
 [[ -1   0  -1  -2]
 [-39   0 -38 -41]
 [ -1   0  -2  -3]]
1
